The lack of an explicit momentum source could cause the biases we are seeing in the SAM simulation. In this notebook, I train a neural network for the momentum source in the same manners as I did for QT and SLI. I also want to see the parametrized source that this gives.

# Imports, Functions and Data Loading

In [ ]:
from toolz import pipe
import uwnet.interface
from uwnet.model import MLP
import xarray as xr

from uwnet.interface import call_with_xr as forward_xr

# define paths for data and nn model
train_data_path = "../data/processed/2018-10-02-ngaqua-subset.nc"
model_path = "../models/4/9.pkl"

# load the model and training data
data = xr.open_dataset(train_data_path)
# mlp = MLP.from_path(model_path)

# Analysis

In [ ]:
# Visualization imports

from ipywidgets import interact, FloatSlider
from gnl.colorblind import colorblind_matplotlib
colorblind_matplotlib()

In [ ]:


def load_and_predict(model_path, data, **kw):
    mlp = MLP.from_path(model_path)
    return forward_xr(mlp, data, **kw)

In [ ]:
diagnosis = load_and_predict(model_path, data)
prediction = load_and_predict(model_path, data, n=1)

In [ ]:
prediction.U.isel(x=0).plot(x='time')

In [ ]:
prediction.UOBS.isel(x=0).plot(x='time')

## Dissipation

In [ ]:
dims = ['x', 'time']
dissip_x = (prediction.FUNN * prediction.U).mean(dims)/(prediction.U**2).mean(dims)


plt.plot(dissip_x.values*86400)
plt.grid()
plt.xlabel('Vertical grid number')

The model is mostly damping in the in the free troposphere, but it is amplifying in the lowest few grid points.

In [ ]:

plt.plot(1/np.abs(dissip_x)/86400)
plt.grid()
ax = plt.gca()

ticks = np.arange(0, dissip_x.shape[0], 5)
ax.set_xticks(ticks)
ax.set_xticklabels(dissip_x.z[ticks].values)
plt.xlabel('Height')
plt.ylabel('Damping/growth time-scale')

The time scales vary from around 1 day in the boundary layer to around 20 in the free troposphere.

## Drift in Mean state

In [ ]:
def plot_mean_drift(prediction):

    fu_mean = prediction.FU.mean(['x', 'time','y'])
    funn_mean = prediction.FUNN.mean(['x', 'time','y'])
    du_obs = (prediction.UOBS[-1]  - prediction.UOBS[0])/(prediction.time[-1]-prediction.time[0])/86400
    du_obs = du_obs.mean(['x', 'y'])

    plt.figure(figsize=(3,6))

    fu_mean.plot(y='z', label='FU')
    funn_mean.plot(y='z', label='FUNN')
    (funn_mean+fu_mean).plot(label='FU-FUNN', y='z')
    (du_obs).plot(label=r'$\Delta U / \Delta t$', y='z')

    a = 2e-5
    plt.xlim([-a, a])
    plt.legend()


In [ ]:
plot_mean_drift(prediction)

Is this problem also in the diagnosis?

In [ ]:
plot_mean_drift(diagnosis)

It is. Perhaps we should penalize the difference in the mean drifts? Would this work batchwise? Perhaps we should also make sure the output of the neural network is reasonably smooth.

# Fixing this

In [ ]:
def plot_mean_drift_file(model_path):
    model = MLP.from_path(model_path)
    diagnosis = forward_xr(model, data)
    plot_mean_drift(diagnosis)
    
    


## Increase seq_length to 40

In [ ]:
plot_mean_drift_file("../models/6/9.pkl")

This helps a little.

## Increase the batch size and number of training steps

In [ ]:
plot_mean_drift_file("../models/7/17.pkl")

## Train for even more epochs

In [ ]:
plot_mean_drift_file("../models/8/0.pkl")

In [ ]:
plot_mean_drift_file("../models/8/15.pkl")

Using too long of sequence length seems to hurt!

## One step prediction

```
python -m uwnet.train with data=data/processed/2018-10-02-ngaqua-subset.nc examples/momentum.yaml batch_size=32 n_epochs=40 seq_length=2 skip=1 lr=.005
```

In [ ]:
plot_mean_drift_file("../models/10/1.pkl")

In [ ]:
plot_mean_drift_file("../models/10/17.pkl")

This leads to even bigger bias.

## Do not add FU

Because FU could be so large

In [ ]:
plot_mean_drift_file("../models/11/35.pkl")

# Analysis

Why are these biases happening? Could it be something with the loss function?

In [ ]:
def get_mom_budget(data):
    dt = 3*3600
    STOR= data.U.diff('time')/dt
    return  xr.Dataset(dict(
        FU=data.FU,
        FUNN=data.FUNN
    )).assign(STOR=STOR, Q3=STOR-data.FU)

    
    
#     data.FUNN.plot()
# path = "../models/11/13.pkl"

diagnosis = forward_xr(MLP.from_path(model_path), data)
mom = get_mom_budget(diagnosis)

In [ ]:
mom.to_array(name='SRC').isel(x=0).plot(col='variable', col_wrap=1, aspect=4, size=2, x='time')

What is going on with the large diagonal disturbances. This does not seem physically realistic. If I recall correctly, these structures are also present in the Q1, and in the raw temperature time series, but they have a smaller amplitude.

In the boundary layer the output of the momentum damping scheme seems reasonable.

In [ ]:
mom.to_array(name='SRC').isel(x=0, z=slice(0,10)).plot(col='variable', col_wrap=1, aspect=4, size=2, x='time')

Here is a zoom-inned picture on this disturbance:

In [ ]:
def get_roi(x):
    return x.isel(x=0).sel(time=slice(106, 108.0))

get_roi(mom).to_array(name='SRC').plot(col='variable', col_wrap=4, aspect=1, size=3, x='time')

 It has a very narrow vertical extent, which is only a couple of vertical levels in size. I don't think the neural network can learn such fine graind vertical structures.
 
 This signal is also shows up in the time series of $\partial_y SLI$, which makes me think this a pressure gradient:

In [ ]:
data.SLI.diff('x').pipe(get_roi).plot(x='time')

If we look in the horizontal direction at day 107, it is clear that the pattern we are seeing is due to the propagation of a large-scale disturbance (near x=0,2e7). 

In [ ]:
data.U.sel(time=107).plot(x='x')

The edge of this wave has incredibly large wind shears. The mixing in these shear zones should have a dipole structure, which we can see in the FUNN time series.

In [ ]:
mom.FUNN.sel(time=107).plot(x='x')

In [ ]:
mom.FUNN.sel(time=107, z=slice(0,2000)).plot(x='x')

This feature is also in the approximate $Q3$, but that data is much noisier.

In [ ]:
mom.Q3.sel(time=107).plot(x='x')

# Questions

**Should this forcing due to the shock-like disturbance be part of the momentum residual budget?** This does not seem like a very physically plausible source of momentum. Apart from this disturbance, the learned momentum forcing seems to be doing good things:

1. In regions with strong convergence and heating, presumably, there is a $cos(3z)-cos(z)$ structure as predicited by Andy's work. this is a dipole structure within the free troposphere.
2. the sign of FUNN in the first 2 levels has an opposite sign as above.